# Analytics For Unstructured Data: Group Assignment #1

## Scraper
https://realpython.com/modern-web-automation-with-python-and-selenium/ 

In [36]:
# Use this code if using Colab to run Selenium

# Make sure to go to Runtime -> Change runtime and set GPU as hardware accelerator

# !kill -9 -1 # Use this line to delete this VM and start a new one.
# The above line deletes all files and folders from the current VM and allocates a new one.

#Selenium is an open-source tool that automates web browsers.
!pip install selenium
!apt-get -q update   #Used to handle installation and removal of softwares and libraries
!apt install -yq chromium-chromedriver #ChromeDriver is a separate executable that Selenium WebDriver uses to control Chrome.
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import nltk
from collections import Counter
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn import manifold
import seaborn as sns
from nltk.tokenize import RegexpTokenizer
import string
import operator
import io
import csv
import re
import decimal
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from itertools import combinations
import shutil
from tempfile import NamedTemporaryFile
#WebDriver is a browser automation framework that works with open source APIs.
#The framework operates by accepting commands, sending those commands to a browser, and interacting with applications.
chrome_options = webdriver.ChromeOptions()
#headless means running chrome with chrome.exe
chrome_options.add_argument('--headless')

chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

zsh:1: command not found: apt-get
The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

cp: /usr/lib/chromium-browser/chromedriver: No such file or directory


In [2]:
driver = webdriver.Chrome(options=chrome_options)

# Create DataFrame
comments = pd.DataFrame(columns = ['Date','user_id','comments'])


#Srape Dates, Usernames, and Comments from most recent 115 pages (about 5000 comments)
for i in range(320,436):

    #address where scraping
    webpage = 'https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p' + str(i)
    driver.get(webpage)

    ids = driver.find_elements(By.XPATH,"//*[contains(@id,'Comment_')]")

    comment_ids = []

    for i in ids:
        comment_ids.append(i.get_attribute('id'))

    #check if there is a blockquote (used in replies to comments) and remove
    for x in comment_ids:
        try:
            element = driver.find_elements(By.XPATH,'//*[@id="' + x +'"]/div/div[3]/div/div[1]/blockquote')[0]
            driver.execute_script("""
                var element = arguments[0];
                element.parentNode.removeChild(element);
                """, element)
        except:
            pass

    for x in comment_ids:

        #Extract dates from for each user on a page
        user_date = driver.find_elements(By.XPATH,'//*[@id="' + x +'"]/div/div[2]/div[2]/span[1]/a/time')[0]
        date = user_date.get_attribute('title')

        #Extract user ids from each user on a page
        userid_element = driver.find_elements(By.XPATH,'//*[@id="' + x +'"]/div/div[2]/div[1]/span[1]/a[2]')[0]
        userid = userid_element.text

        #Extract Message for each user on a page
        user_message = driver.find_elements(By.XPATH,'//*[@id="' + x +'"]/div/div[3]/div/div[1]')[0]

        comment = user_message.text


        #Adding date, userid and comment for each user in a dataframe
        comments.loc[len(comments)] = [date,userid,comment]

In [ ]:
#Nulls don't play well with the tokenizer, so drop nulls
comments.dropna(inplace=True)

#change the location as per the file destination you want it to be
comments.to_csv('data/Edmunds_scraped2.0.csv', index=False)

In [28]:
#If working from CSV instead of scraping originally, read in CSV
comments = pd.read_csv('data/Edmunds_scraped2.0.csv')

## Task A: Zipf's Law

In [30]:
# raw word frequencies
import csv
import re
import string
from collections import defaultdict
from nltk.corpus import stopwords
import nltk
import io

# Download stopwords from the NLTK package
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Input and output filenames
input_filename = 'data/Edmunds_scraped2.0.csv'  # Input file
final_filename = 'data/final.csv'  # Intermediate file without the column header
word_freq_output = 'data/word_freq.csv'  # Output file for word frequencies

# Function to clean and tokenize sentences
def clean_and_tokenize(sentence):
    """
    Cleans a given sentence by removing punctuation and stopwords, converting text to lowercase,
    and tokenizing the remaining words.
    """
    # Remove punctuation and convert text to lowercase
    sentence = re.sub(f'[{re.escape(string.punctuation)}]', '', sentence.lower())

    # Tokenize and remove stopwords
    return [word for word in sentence.split() if word not in stop_words]

# Step 1: Remove header from the input CSV and create a new file without it
def remove_header(input_file, output_file):
    """
    Reads the input CSV file, removes the header, and writes the remaining rows
    into a new output file.
    """
    file = pd.read_csv(input_file, header = None, skiprows = 1)
    file.to_csv(output_file, index = False)

# Step 2: Extract and clean sentences from the text
def extract_sentences(file):
    """
    Extracts text data from the third column of the CSV file, splits it into sentences,
    and cleans each sentence by removing punctuation and stopwords.
    The result is a list with 3 levels: message, sentences, and words
    """
    df = pd.read_csv(file)
    posts = df.iloc[:,2]
    sentences = []
    sentences_clean = []
    for post in posts:
        sentences.extend(re.split('[?.!]', post))
    for sentence in sentences:
        cleaned_tokens = clean_and_tokenize(sentence)
        if cleaned_tokens:
            sentences_clean.append(cleaned_tokens)
    return sentences_clean


# Step 3: Calculate word frequencies
def calculate_word_frequencies(sentences):
    """
    Calculates the frequency of each word in the given list of cleaned sentences.
    """
    freqs = {}
    total_words = 0
    for sentence in sentences:
        for word in sentence:
            freqs[word] = freqs.get(word, 0) + 1
            total_words += 1
    return freqs

# Step 4: Write word frequencies to CSV
def write_word_frequencies(word_freq, output_file):
    """
    Writes the word frequencies to the specified CSV file.
    """
    word_freq_df = pd.DataFrame(word_freq.items(), columns = ["Word", "Frequency"])
    word_freq_df = word_freq_df.sort_values(by = "Frequency", ascending = False)
    word_freq_df.to_csv(output_file, index=False)
    print(f"Word frequencies written to {output_file}")

# Main function to run all steps
def main():
    # Step 1: Remove header
    remove_header(input_filename, final_filename)
    
    # Step 2: Extract and clean sentences
    sentences = extract_sentences(final_filename)
    
    # Step 3: Calculate word frequencies
    word_freq = calculate_word_frequencies(sentences)
    
    # Step 4: Write word frequencies to CSV
    write_word_frequencies(word_freq, word_freq_output)

if __name__ == "__main__":
    main()


[nltk_data] Downloading package stopwords to /Users/kenne/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Word frequencies written to data/word_freq.csv


In [31]:
pd.read_csv('data/word_freq.csv')

,Word,Frequency
0,car,2715
1,like,1739
2,would,1738
3,cars,1655
4,one,1599
...,...,...
20705,clatter,1
20706,allinall,1
20707,pets,1
20708,wnav,1


## Task B: Frequencies

### Find and Replace

In [33]:
import csv
import re
import shutil
from tempfile import NamedTemporaryFile

# Filepaths
output_file = 'data/find_and_replace.csv'  # The file where the modified data will be stored
input_file = 'data/Edmunds_scraped2.0.csv'     # The file containing the original data
replacement_file = 'data/car_models_and_brands.csv'  # The file containing original and replacement words

# Create a temporary file to write the changes before moving it to the final location
tempfile = NamedTemporaryFile(mode='w', delete=False, newline='', encoding='utf-8')

def load_replacements(replacement_file):
    """
    Load word replacements from a CSV file into a dictionary.
    The right column contains words to be replaced by the corresponding words in the left column.
    """
    replacements = {}
    with open(replacement_file, 'r', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in reader:
            original, replacement = row[1].lower(), row[0].lower()  # Ensure lowercase comparison
            replacements[original] = replacement
    return replacements

def replace_words_in_text(text, replacements):
    """
    Replace words in the input text according to the replacements dictionary.
    """
    words = text.split()  # Split text into words
    new_words = []
    for word in words:
        # Remove punctuation from the word
        word_clean = re.sub(r'[^\w\s]', '', word).lower() 
        # Replace word if it's in the replacements dictionary
        if word_clean in replacements:
            new_word = replacements[word_clean]
            # Preserve original word's punctuation and case
            new_words.append(re.sub(word_clean, new_word, word, flags=re.IGNORECASE))
        else:
            new_words.append(word)
    return ' '.join(new_words)

def process_file(input_file, output_file, replacements):
    """
    Read the input file, perform word replacements, and write the modified content to the output file.
    """
    with open(input_file, 'r', encoding='utf-8') as infile, tempfile:
        reader = csv.reader(infile, delimiter=',', quotechar='"')
        writer = csv.writer(tempfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        
        # Write the header if present
        header = next(reader)
        writer.writerow(header)

        for row in reader:
            new_row = [replace_words_in_text(col, replacements) for col in row]
            writer.writerow(new_row)

    # Move the temp file to the final output location
    shutil.move(tempfile.name, output_file)

def main():
    # Load the replacement words from the replacement CSV file
    replacements = load_replacements(replacement_file)
    
    # Process the input file and apply the replacements
    process_file(input_file, output_file, replacements)

if __name__ == "__main__":
    main()

### Frequency Counts

In [ ]:
# redo the frequency counts so you only count each word once per message


## Task C: Lift Calculation

In [37]:
# Initialize global variables and data structures
df_lift = pd.DataFrame(columns=['word1', 'word2', 'lift_value'])  # To store lift values
word_frequency = {}  # Dictionary to store word frequency in posts
word_pair_frequency = defaultdict(dict)  # Dictionary to store word pair co-occurrence frequency
results_dict = {}  # Dictionary to store results with lift values for word pairs
file_length = 0  # Number of rows in the input file
itr = 0  # Row iterator for the lift DataFrame

# File paths
input_file = 'data/find_and_replace.csv'  # Input data file
pair_keys_file = 'edmunds_pair_keys.txt'  # File containing the words to calculate lift
output_lift_values = 'data/Lift_Values.csv'  # Output file for lift values
output_lift_matrix = 'data/Lift_Matrix.csv'  # Output file for lift matrix

# Load NLTK stopwords
stop_words = set(stopwords.words('english'))

# Function to clean and tokenize text (removes punctuation and stopwords)
def clean_text(text):
    """
    Cleans a given text by removing punctuation, converting it to lowercase,
    and tokenizing it, ignoring any stopwords.
    """
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize text
    tokens = word_tokenize(text)

    # Remove stopwords
    cleaned_tokens = [word for word in tokens if word not in stop_words]

    return cleaned_tokens

# Step 1: Load the words from the edmunds_pair_keys.txt file and generate all pairs
def load_word_pairs(filename):
    """
    Loads words from a file where words are comma-separated in each row.
    Returns a list of all possible word pairs for each row.
    """
    word_pairs = []
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            # Generate all possible word pairs from each row
            pairs = list(combinations(row, 2))
            word_pairs.extend(pairs)

    return word_pairs

# Step 2: Process the input CSV file to extract posts and clean the text
def process_input_file(input_filename):
    """
    Processes the input CSV file to extract and clean posts. Each post is tokenized,
    cleaned of punctuation and stopwords, and stored in a list.
    """
    posts = []
    global file_length
    df = pd.read_csv(input_filename)  # Load the CSV file into a DataFrame

    # Assuming 'comments' is the column that contains the text
    for index, row in df.iterrows():
        cleaned_post = clean_text(row['comments'])  # Clean and tokenize the post
        posts.append(cleaned_post)

    file_length = len(df)  # Get the total number of rows
    return posts

# Step 3: Calculate word frequencies and word pair co-occurrences (distance ≥ 5 words)
def calculate_frequencies(posts):
    """
    Calculates the frequency of individual words and word pairs within the posts.
    Updates the global word_frequency and word_pair_frequency dictionaries.
    Only considers word pairs that are 5 or more words apart.
    """
    global word_frequency, word_pair_frequency

    for post in posts:
        word_positions = {}  # Dictionary to track positions of each word

        # Track word positions
        for idx, word in enumerate(post):
            if word not in word_positions:
                word_positions[word] = []
            word_positions[word].append(idx)

        # Count word frequencies
        unique_words = set(post)  # Track unique words in the post to avoid double counting
        for word in unique_words:
            word_frequency[word] = word_frequency.get(word, 0) + 1

        # Count word pair co-occurrences with distance check
        for word1 in word_positions:
            for word2 in word_positions:
                if word1 != word2:
                    # Check if the words are 5 or more positions apart
                    for pos1 in word_positions[word1]:
                        for pos2 in word_positions[word2]:
                            if abs(pos1 - pos2) >= 5:
                                word_pair_frequency[word1][word2] = word_pair_frequency[word1].get(word2, 0) + 1
# Step 4: Calculate the lift between word pairs
def calculate_lift(word_pairs):
    """
    Calculates the lift between word pairs using the formula:
    Lift(word1, word2) = P(word1 AND word2) / (P(word1) * P(word2))
    Lift is written to the lift values CSV and stored in a DataFrame for further processing.
    """
    global itr

    for word1, word2 in word_pairs:
        # Get the frequency of word1, word2, and their co-occurrence
        freq_word1 = word_frequency.get(word1, 0)
        freq_word2 = word_frequency.get(word2, 0)
        co_occurrence = word_pair_frequency.get(word1, {}).get(word2, 0)

        # Calculate probabilities
        p_word1 = freq_word1 / file_length if freq_word1 else 0
        p_word2 = freq_word2 / file_length if freq_word2 else 0
        p_word1_and_word2 = co_occurrence / file_length if co_occurrence else 0

        # Avoid division by zero
        if p_word1 > 0 and p_word2 > 0:
            lift_value = p_word1_and_word2 / (p_word1 * p_word2) if (p_word1 * p_word2) > 0 else 0
        else:
            lift_value = 0

        # Store lift value in DataFrame
        df_lift.loc[itr] = [word1, word2, lift_value]
        itr += 1

# Step 5: Write lift values and matrix to CSV
def save_results():
    """
    Writes the calculated lift values to a CSV file and also generates a lift matrix,
    saving it to another CSV.
    """
    # Save lift values DataFrame to CSV
    df_lift.to_csv(output_lift_values, index=False)

    # Generate lift matrix
    lift_matrix = pd.pivot_table(df_lift, values='lift_value', index='word1', columns='word2', fill_value=0)
    lift_matrix.to_csv(output_lift_matrix)

# Main function to run all steps
def main():
    # Step 1: Load word pairs
    word_pairs = load_word_pairs(pair_keys_file)

    # Step 2: Process the input file to extract posts
    posts = process_input_file(input_file)

    # Step 3: Calculate frequencies
    calculate_frequencies(posts)

    # Step 4: Calculate lift values
    calculate_lift(word_pairs)

    # Step 5: Save results
    save_results()

# Run the script
if __name__ == "__main__":
    main()

## Task D: MDS Map
https://www.geeksforgeeks.org/sklearn-multi-dimensional-scaling-mds-python-implementation-from-scratch/

https://github.com/tanyakuznetsova/Multidimensional-Scaling-of-European_Cities/blob/main/Multidimensional_scaling_of_European_Cities.ipynb

## Task E: Insights from C and D

## Task F: Car Attributes

## Task G: Advice from F

## Task H: Aspirational Brands